In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# change the path to where your notebook is located
%cd "gdrive/My Drive/nli"

/content/gdrive/My Drive/nli


In [ ]:
!pip install datasets
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

In [ ]:
# Run it only once
from datasets import load_dataset
dataset = load_dataset("multi_woz_v22")

Extracting data files:   0%|          | 0/22 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Run it only once
dataset.save_to_disk("dataset.hf")

Saving the dataset (0/1 shards):   0%|          | 0/8437 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict
dataset = DatasetDict.load_from_disk("dataset.hf")

In [ ]:
#first version of the dataframe for the purpose of dialog classification, where given a utterance we directly map it to its dialog act.
def make_df(split):

  frame = []
  conjunto = set()

  for d,dial in enumerate(dataset[split]):
      # print(dial)
      # print(dial['dialogue_id'])
      speaker_str = {0: 'User', 1: 'Agent'}
      turns = dial['turns']
      #if some dialogue does not contain any restaurant-hotel domain then you just skip that because we are only focusing on hotel and restaurant
      if not any(set(dial['turns']['frames'][turn_id]['service']).intersection(['hotel', 'restaurant']) for turn_id,utt in enumerate(dial['turns']['utterance'])):
        continue
      for turn_id,utt in enumerate(turns['utterance']):
          visited = set()
          speaker = speaker_str[turns['speaker'][turn_id]]
          dialogue_acts = turns['dialogue_acts'][turn_id]['dialog_act']['act_type']
          acts_dialog = []
          for act in dialogue_acts:
            if 'Attraction' in act or 'Train' in act or 'Hospital' in act or 'Taxi' in act or 'Police' in act:
              act_single = act.split('-')
              if act_single[0] not in visited:
                acts_dialog.append(act_single[0])
                conjunto.add(act_single[0])
                visited.add(act_single[0])
            else:
              # print(f"{act} HERE")/
              acts_dialog.append(act)
              conjunto.add(act)
          if(len(acts_dialog)) > 0:
            frame.append([dial['dialogue_id'], utt, turn_id, speaker, acts_dialog])

  frame = np.array(frame, dtype=object)
#we export a pandas dataframe
  return pd.DataFrame(frame, columns=['conv_id', 'utterance', 'turn_id', 'speaker', 'acts']), conjunto



In [ ]:
train_df = make_df('train')
test_df = make_df('test')
val_df = make_df('validation')

In [ ]:
train_df

(             conv_id                                          utterance  \
 0      PMUL4398.json  i need a place to dine in the center thats exp...   
 1      PMUL4398.json  I have several options for you; do you prefer ...   
 2      PMUL4398.json  Any sort of food would be fine, as long as it ...   
 3      PMUL4398.json  There is an Afrian place named Bedouin in the ...   
 4      PMUL4398.json  Sounds good, could I get that phone number? Al...   
 ...              ...                                                ...   
 90646  WOZ20469.json  nandos serves portuguese food and in the cheap...   
 90647  WOZ20469.json  I would like the address of Nandos restaurant,...   
 90648  WOZ20469.json  Nandos is located in the south part of the cit...   
 90649  WOZ20469.json                                Thank you, goodbye.   
 90650  WOZ20469.json                                Thank you good bye.   
 
       turn_id speaker                                     acts  \
 0           0    U

In [ ]:
#this code generates the concatenated structure that we are using to feed our model. prev_user_utt+dialog_act+user_utt
def concatenate_df(df):

  df['acts_no_list'] = df['acts'].apply(lambda x: ''.join(map(str, x)))
  # Create the concatenated utterance with special separators between separate utterances
  df['concatenated_utterance'] = (
      df['utterance'].shift(1) + '|' + df['acts_no_list'].shift(1) + '|' +
      df['utterance'] + '|' +
      df['utterance'].shift(-1) + '|' + df['acts_no_list'].shift(-1)
  )

  # Deal with the special cases of first and last row
  df.at[0, 'concatenated_utterance'] = (
      df['utterance'][0] + '|' + df['utterance'][1] + '|' + df['acts_no_list'][1]
  )
  df.at[df.shape[0] - 1, 'concatenated_utterance'] = (
      df['utterance'][df.shape[0] - 2] + '|' +
      df['acts_no_list'][df.shape[0] - 2] + '|' + df['utterance'][df.shape[0] - 1]
  )

  return df

In [ ]:
train_df_conc=concatenate_df(train_df[0])
test_df_conc=concatenate_df(test_df[0])
val_df_conc=concatenate_df(val_df[0])

In [ ]:
train_df_conc

,conv_id,utterance,turn_id,speaker,acts,acts_no_list,concatenated_utterance
0,PMUL4398.json,i need a place to dine in the center thats exp...,0,User,[Restaurant-Inform],Restaurant-Inform,i need a place to dine in the center thats exp...
1,PMUL4398.json,I have several options for you; do you prefer ...,1,Agent,"[Restaurant-Inform, Restaurant-Select]",Restaurant-InformRestaurant-Select,i need a place to dine in the center thats exp...
2,PMUL4398.json,"Any sort of food would be fine, as long as it ...",2,User,[Restaurant-Request],Restaurant-Request,I have several options for you; do you prefer ...
3,PMUL4398.json,There is an Afrian place named Bedouin in the ...,3,Agent,[Restaurant-Inform],Restaurant-Inform,"Any sort of food would be fine, as long as it ..."
4,PMUL4398.json,"Sounds good, could I get that phone number? Al...",4,User,"[Hotel-Inform, Restaurant-Request]",Hotel-InformRestaurant-Request,There is an Afrian place named Bedouin in the ...
...,...,...,...,...,...,...,...
90646,WOZ20469.json,nandos serves portuguese food and in the cheap...,5,Agent,"[Restaurant-Recommend, general-reqmore]",Restaurant-Recommendgeneral-reqmore,It doesn't matter.|Restaurant-Inform|nandos se...
90647,WOZ20469.json,"I would like the address of Nandos restaurant,...",6,User,"[Restaurant-Inform, Restaurant-Request]",Restaurant-InformRestaurant-Request,nandos serves portuguese food and in the cheap...
90648,WOZ20469.json,Nandos is located in the south part of the cit...,7,Agent,"[Restaurant-Inform, general-reqmore]",Restaurant-Informgeneral-reqmore,"I would like the address of Nandos restaurant,..."
90649,WOZ20469.json,"Thank you, goodbye.",8,User,[general-bye],general-bye,Nandos is located in the south part of the cit...


In [ ]:
def word_to_num(df):
  dummies = pd.get_dummies(df['acts'].apply(pd.Series).stack()).sum(level=0)

  # Concatenate the dummies DataFrame with the original DataFrame
  result = pd.concat([df, dummies], axis=1)

  # Drop the 'acts' column
  result = result.drop('acts', axis=1)
  return result

In [ ]:
train_df = word_to_num(train_df_conc)
val_df = word_to_num(val_df_conc)
test_df = word_to_num(test_df_conc)

<ipython-input-111-5f656858e63e>:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df['acts'].apply(pd.Series).stack()).sum(level=0)
<ipython-input-111-5f656858e63e>:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df['acts'].apply(pd.Series).stack()).sum(level=0)
<ipython-input-111-5f656858e63e>:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df['acts'].apply(pd.Series).stack()).sum(level=0)


In [ ]:
train_df

,conv_id,utterance,turn_id,speaker,acts_no_list,concatenated_utterance,Attraction,Booking-Book,Booking-Inform,Booking-NoBook,...,Restaurant-Recommend,Restaurant-Request,Restaurant-Select,Taxi,Train,general-bye,general-greet,general-reqmore,general-thank,general-welcome
0,PMUL4398.json,i need a place to dine in the center thats exp...,0,User,Restaurant-Inform,i need a place to dine in the center thats exp...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PMUL4398.json,I have several options for you; do you prefer ...,1,Agent,Restaurant-InformRestaurant-Select,i need a place to dine in the center thats exp...,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,PMUL4398.json,"Any sort of food would be fine, as long as it ...",2,User,Restaurant-Request,I have several options for you; do you prefer ...,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,PMUL4398.json,There is an Afrian place named Bedouin in the ...,3,Agent,Restaurant-Inform,"Any sort of food would be fine, as long as it ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,PMUL4398.json,"Sounds good, could I get that phone number? Al...",4,User,Hotel-InformRestaurant-Request,There is an Afrian place named Bedouin in the ...,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90646,WOZ20469.json,nandos serves portuguese food and in the cheap...,5,Agent,Restaurant-Recommendgeneral-reqmore,It doesn't matter.|Restaurant-Inform|nandos se...,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
90647,WOZ20469.json,"I would like the address of Nandos restaurant,...",6,User,Restaurant-InformRestaurant-Request,nandos serves portuguese food and in the cheap...,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
90648,WOZ20469.json,Nandos is located in the south part of the cit...,7,Agent,Restaurant-Informgeneral-reqmore,"I would like the address of Nandos restaurant,...",0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
90649,WOZ20469.json,"Thank you, goodbye.",8,User,general-bye,Nandos is located in the south part of the cit...,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
def drop_agent(df):
  return df[df['speaker'] != 'Agent']

In [ ]:
train_df = drop_agent(train_df)
val_df = drop_agent(val_df)
test_df = drop_agent(test_df)

In [ ]:
train_df.drop(columns=['turn_id', 'speaker','utterance','acts_no_list'], inplace=True)
val_df.drop(columns=['turn_id', 'speaker','utterance','acts_no_list'], inplace=True)
test_df.drop(columns=['turn_id', 'speaker','utterance','acts_no_list'], inplace=True)

In [ ]:
train_df

,conv_id,concatenated_utterance,Attraction,Booking-Book,Booking-Inform,Booking-NoBook,Booking-Request,Hospital,Hotel-Inform,Hotel-NoOffer,...,Restaurant-Recommend,Restaurant-Request,Restaurant-Select,Taxi,Train,general-bye,general-greet,general-reqmore,general-thank,general-welcome
0,PMUL4398.json,i need a place to dine in the center thats exp...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PMUL4398.json,I have several options for you; do you prefer ...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,PMUL4398.json,There is an Afrian place named Bedouin in the ...,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
6,PMUL4398.json,Bedouin's phone is 01223367660. As far as hote...,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,PMUL4398.json,"Sure, when would you like that reservation?|Bo...",0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90641,WOZ20469.json,"Goodbye.|general-bye|Hello, I am looking for a...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90643,WOZ20469.json,It looks like there are no German restaurants ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90645,WOZ20469.json,There are two restaurants in the cheap price r...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90647,WOZ20469.json,nandos serves portuguese food and in the cheap...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
labels = [label for label in train_df.columns if label not in ['conv_id', "concatenated_utterance"]]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Attraction',
 'Booking-Book',
 'Booking-Inform',
 'Booking-NoBook',
 'Booking-Request',
 'Hospital',
 'Hotel-Inform',
 'Hotel-NoOffer',
 'Hotel-Recommend',
 'Hotel-Request',
 'Hotel-Select',
 'Police',
 'Restaurant-Inform',
 'Restaurant-NoOffer',
 'Restaurant-Recommend',
 'Restaurant-Request',
 'Restaurant-Select',
 'Taxi',
 'Train',
 'general-bye',
 'general-greet',
 'general-reqmore',
 'general-thank',
 'general-welcome']

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def preprocessing(examples):
    # take a batch of texts
    text = examples["concatenated_utterance"]
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}

    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))

    # fill numpy array
    for idx, label in enumerate(labels):
        # Check if the label is present in the batch
        if label in labels_batch:
            labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
encoded_train = train_dataset.map(preprocessing, batched=True, remove_columns=train_dataset.column_names)
encoded_test = test_dataset.map(preprocessing, batched=True, remove_columns=test_dataset.column_names)
encoded_val = val_dataset.map(preprocessing, batched=True, remove_columns=val_dataset.column_names)

Map:   0%|          | 0/44994 [00:00<?, ? examples/s]

Map:   0%|          | 0/5694 [00:00<?, ? examples/s]

Map:   0%|          | 0/5827 [00:00<?, ? examples/s]

In [ ]:
#create the tensorial structure once the dataset is preprocessed to the adequate structure of the data for the BERT model training
encoded_train.set_format("torch")
encoded_test.set_format("torch")
encoded_val.set_format("torch")

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("roberta-base",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 16
metric_name = "f1"

from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    "confs",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
#this code was taken online and it is basically used for computing the metrics during the training
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    print(f'y_true = {y_true}')
    print(f'y_pred = {y_pred}')
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def calculate_performance_metrics(predictions: EvalPrediction):
    model_predictions = predictions.predictions[0] if isinstance(predictions.predictions, tuple) else predictions.predictions
    metrics_result = evaluate_multi_label_metrics(
        model_predictions=model_predictions,
        true_labels=predictions.label_ids)
    return metrics_result

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=tokenizer,
    compute_metrics=calculate_performance_metrics
)

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.020000,0.018338,0.939419,0.965217,0.905955
2,0.014800,0.014324,0.952758,0.973898,0.922602
3,0.011600,0.013329,0.955401,0.974179,0.926892
4,0.009600,0.013075,0.955798,0.975959,0.926892


y_true = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
y_pred = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
y_true = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
y_pred = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
y_true = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
y_pred = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
y_true = [

TrainOutput(global_step=11252, training_loss=0.020250399829737028, metrics={'train_runtime': 4578.6709, 'train_samples_per_second': 39.307, 'train_steps_per_second': 2.457, 'total_flos': 1.1840757257355264e+16, 'train_loss': 0.020250399829737028, 'epoch': 4.0})

In [ ]:
model.save_pretrained('roberta_dialog_classification')


In [ ]:
model.to('cpu')

In [ ]:
from transformers import AutoModelForSequenceClassification

# Replace 'path_to_your_model_directory' with the actual path to your saved model directory
model_directory = 'gdrive/My Drive/nli/roberta_dialog_classification'

# Load the pre-trained BERT model
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_directory)


In [ ]:
model =  AutoModelForSequenceClassification.from_pretrained('roberta_dialog_classification')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def predict(utt):
  inputs = tokenizer(utt, return_tensors="pt")
  #inputs.to('cuda')

  with torch.no_grad():
      logits = model(**inputs).logits

  predictions = torch.argmax(logits, dim=1)
  #print(predictions)
  predicted_class=id2label[predictions.item()]
  return predicted_class


In [ ]:
for_scores = train_df_conc[train_df_conc['speaker'] == 'User']

In [ ]:
test = for_scores.to_numpy()

In [ ]:
true_vals = test[:, 4]
true_vals

array([list(['Restaurant-Inform']), list(['Restaurant-Request']),
       list(['Hotel-Inform', 'Restaurant-Request']), ...,
       list(['Restaurant-Inform']),
       list(['Restaurant-Inform', 'Restaurant-Request']),
       list(['general-bye'])], dtype=object)

In [ ]:
pred_vals = [predict(row[1], model) for row in test]

In [ ]:
pred_vals

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=list(labels))
y_true_list = [mlb.fit_transform([true_labels]) for true_labels in true_vals]
y_pred_list = [mlb.transform([predicted_labels]) for predicted_labels in pred_vals]


In [ ]:
y_true_list

y_true = []
y_pred = []

for i in range(len(y_true_list)):
  print()
  y_true.append(y_true_list[i][0])
  y_pred.append(y_pred_list[i][0])
print(classification_report(y_true, y_pred,target_names=list(labels)))

Streaming output truncated to the last 5000 lines.























































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, target_names=list(labels))

# Print the report
print(report)

                      precision    recall  f1-score   support

          Attraction       0.76      0.74      0.75      4500
        Booking-Book       0.00      0.00      0.00         0
      Booking-Inform       0.00      0.00      0.00         0
      Booking-NoBook       0.00      0.00      0.00         0
     Booking-Request       0.00      0.00      0.00         0
            Hospital       0.00      0.00      0.00        19
        Hotel-Inform       0.94      0.84      0.89     11521
       Hotel-NoOffer       0.00      0.00      0.00         0
     Hotel-Recommend       0.00      0.00      0.00         0
       Hotel-Request       0.72      0.41      0.52      2107
        Hotel-Select       0.00      0.00      0.00         0
              Police       0.00      0.00      0.00        12
   Restaurant-Inform       0.95      0.80      0.87     11063
  Restaurant-NoOffer       0.00      0.00      0.00         0
Restaurant-Recommend       0.00      0.00      0.00         0
  Resta

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
